In [ ]:
#CODE FOR INITIALIZING REWARD TABLE
import pickle
from pprint import pprint as pp
import datetime
import glob
import random
import os
from textblob import TextBlob
import pandas as pd
import sys
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from collections import Counter
stop = set(stopwords.words('english'))

rel_path = "data/new_data/"
abs_data_path = os.path.join(os.getcwd(), rel_path)

rel_path = "strategies.pkl"

abs_strategies_path = os.path.join(os.getcwd(), rel_path)
plot_dict={}

N_TURN = 3
list1=[]
bot_strategies=[]
learning_rate=0.9
discount_factor=0.9
epsilon=0.1

user_utterances=[]
strategies_list=[]

evaluation_dict={}
user_lines=[]
all_state_variables=['qs+l',"qs-l","qs'l","qs+l'","qs-l'","qs'l'","q's+l","q's-l","q's'l","q's+l'","q's-l'","q's'l'"]
#state action table
R_table=pd.DataFrame.from_items([('qs+l', [0.0, 0.0, 0.0,0.0,0.0,0.0]), ("qs-l", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("qs'l", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("qs+l'", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                           ("qs-l'", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("qs'l'", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q's+l", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q's-l", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                            ("q's'l", [0.0, 0.0, 0.0,0.0,0.0,0.0]), ("q's+l'", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q's-l'", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q's'l'", [0.0, 0.0, 0.0,0.0,0.0,0.0])]
,
                                            orient='index', 
                                          columns=[ 'Answer.change', 'Answer.elaboration','Answer.continuation',
                                                   'Answer.joking'])

#temporary dictionary which stores the count of the recorded utterance till date,this is used for calculating the average to be stored in state table
temp_dict={'qs+l':0,"qs-l":0,"qs'l":0,"qs+l'":0,"qs-l'":0,"qs'l'":0,"q's+l":0,"q's-l":0,"q's'l":0,"q's+l'":0,"q's-l'":0,"q's'l'":0}

Q_table=pd.DataFrame.from_items([('qs+l', [0.0, 0.0, 0.0,0.0,0.0,0.0]), ("qs-l", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("qs'l", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("qs+l'", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                           ("qs-l'", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("qs'l'", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q's+l", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q's-l", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                            ("q's'l", [0.0, 0.0, 0.0,0.0,0.0,0.0]), ("q's+l'", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q's-l'", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q's'l'", [0.0, 0.0, 0.0,0.0,0.0,0.0])]
,
                                            orient='index', 
                                          columns=[ 'Answer.change', 'Answer.elaboration','Answer.continuation',
                                                   'Answer.joking'])
R_table_count={ 'Answer.change':0, 'Answer.elaboration':0,'Answer.continuation':0,
                                                   'Answer.joking':0}
beginning_dict={'Answer.change':0, 'Answer.elaboration':0,'Answer.continuation':0,'Answer.joking':0}
beginning_count={'Answer.change':0, 'Answer.elaboration':0,'Answer.continuation':0,'Answer.joking':0}
def count1(sentence):
    list1=sentence.lower().split()
    return (len(list1))

def diverse_words(total_words,unique_words):
    return total_words/unique_words


def lexical_diversity(sentence):
    #print(sentence)
    #print([i for i in sentence.lower().split() if i not in stop])
    counter_dict={}
    dict=Counter(sentence.split())
    list1=dict.keys()
    unique_words=len(list1)
    total_words=count1(sentence)
    return diverse_words(total_words,unique_words)
#utility functions
def check_strategy(response):
    for key, value in strategies_dict.items():
        if response in value:
            #print(key)
            break
            
    return key
def check_strategies(response):
    #print(response)
    temp_list=response.split(":")
    temp_list2=temp_list[0].split('_')
    return temp_list2[1]
    

def get_utterance(line):
    temp_list=line.split(":")
    value=temp_list[3]
    if "." in value:
        value=value.replace("."," ")
    if "," in value:
        value=value.replace(","," ")
    return value

def max_beginning():
    value=''
    max=0
    for column in Q_table:
        for values in all_state_variables:
            if "b" in values:
                if Q_table.at[values,column]>max:
                    max=Q_table.at[values,column]
                    #print(max)
                    value=column
    return value
def populate_evaluation(line):
    list1=line.split(',')
    for values in list1:
        if values is not '':
            list2=values.split('=')
            evaluation_dict[list2[0]]=list2[1]

#utility functions
def next_state(utterance_no):
    if not utterance_no==len(user_utterances)-1:
        return user_utterances[utterance_no+1]
    else:
        return 0
    
def max_q_value(utterance):
    max=0
    for column in Q_table:
        if Q_table.at[utterance,column]>max:
            max=Q_table.at[utterance,column]
    return max
max_strat=''
def max_strategy(utterance):
    value=''
    max=0
    for column in Q_table:
        if Q_table.at[utterance,column]>max:
            max=Q_table.at[utterance,column]
            value=column
    return value

def training(indexes):
    
    for line in user_utterances:
        next_utterance=next_state(indexes)
        temp_dict[user_utterances[indexes]]=temp_dict[user_utterances[indexes]]+1
        prev_value=Q_table.at[user_utterances[indexes],strategies_list[indexes]]
        #print(user_utterances[indexes],strategies_list[indexes],Q_table.at[user_utterances[indexes],strategies_list[indexes]])
        if next_utterance is not 0:
            
            Q_table.at[user_utterances[indexes],strategies_list[indexes]]=((temp_dict[user_utterances[indexes]]-1)*Q_table.at[user_utterances[indexes],strategies_list[indexes]]+Q_table.at[user_utterances[indexes],strategies_list[indexes]]+learning_rate*(R_table.at[user_utterances[indexes],strategies_list[indexes]] + discount_factor*max_q_value(next_utterance)-Q_table.at[user_utterances[indexes],strategies_list[indexes]]))/temp_dict[user_utterances[indexes]] 
        else:
            Q_table.at[user_utterances[indexes],strategies_list[indexes]]=R_table.at[user_utterances[indexes],strategies_list[indexes]]
        if(user_utterances[indexes]=="q's+l'" and strategies_list[indexes]=="Answer.sarcastic"):
            
            plot_dict[len(plot_dict)]=Q_table.at[user_utterances[indexes],strategies_list[indexes]]
list1=[]
user_utterances=[]
bot_strategies=[]

def calculate_beginning_reward(strategy,next_utterance):
    beginning_count[strategy]=beginning_count[strategy]+1
    #print(strategy,beginning_dict[strategy],max_q_value(next_utterance),next_utterance)
    beginning_dict[strategy]=(((beginning_count[strategy]-1)*beginning_dict[strategy])+max_q_value(next_utterance)+beginning_dict[strategy])/beginning_count[strategy]
    
    
def plot():
    print(len(plot_dict))
    x_val = []
    x_val=list(plot_dict.keys())
    y_val=[]
    y_val=list(plot_dict.values())
    

    
    plt.plot(y_val)
    plt.plot(y_val,'or')
    plt.xlabel("q's+l' Answer.sarcastic")
    print(plot_dict)
    plt.show()
    
def question_or_not(words):
    if "?" not in words:
        q="q'"
    else:
        q="q"
    
    return q

def count_no_words(words):
    no=len(words.split())
    if no>=10:
        
        l="l"
    else:
        
        l="l'"
    return l

def sentiment(words):
    blob = TextBlob(words)
    senti=blob.sentiment
    if senti.polarity<0.2 and senti.polarity>-0.2:
        s="s'"
    else:
        if senti.polarity>0:
            s="s+"
        else:
            s="s-"
    return s
    


def beginning_end(line,list):
    length=len(list)
    if list.index(line)==0:
        
        var="b"
    else:
        
        var="e"
    return var

def rows_utterance(list1):
    for values in list1:
        if question_or_not(values)=="q":
            if sentiment(values)=="s+":
                if count_no_words(values)=="l":
                    result="qs+l"
                else:
                    result="qs+l'"
            elif sentiment(values)=="s-":
                if count_no_words(values)=="l":
                    result="qs-l"
                else:
                    result="qs-l'"
            else:
                if count_no_words(values)=="l":
                    result="qs'l"
                else:
                    result="qs'l'"
        else:
            if sentiment(values)=="s+":
                if count_no_words(values)=="l":
                    result="q's+l"
                else:
                    result="q's+l'"
            elif sentiment(values)=="s-":
                if count_no_words(values)=="l":
                    result="q's-l"
                else:
                    result="q's-l'"
            else:
                if count_no_words(values)=="l":
                    result="q's'l"
                else:
                    result="q's'l'"
        user_utterances.append(result)
    return user_utterances

def single_utterance(values):
    
	if question_or_not(values)=="q":
		if sentiment(values)=="s+":
			if count_no_words(values)=="l":
				result="qs+l"
			else:
				result="qs+l'"
		elif sentiment(values)=="s-":
			if count_no_words(values)=="l":
				result="qs-l"
			else:
				result="qs-l'"
		else:
			if count_no_words(values)=="l":
				result="qs'l"
			else:
				result="qs'l'"
	else:
		if sentiment(values)=="s+":
			if count_no_words(values)=="l":
				result="q's+l"
			else:
				result="q's+l'"
		elif sentiment(values)=="s-":
			if count_no_words(values)=="l":
				result="q's-l"
			else:
				result="q's-l'"
		else:
			if count_no_words(values)=="l":
				result="q's'l"
			else:
				result="q's'l'"
	return result

#_________________

In [ ]:
#CODE FOR EXTRACTING VALUES FROM REWARD TABLE AND LEARNING FROM IT

user_utterances=[]

for filename in os.listdir(abs_data_path):
    
    file = open(abs_data_path+filename,'r')
    
    strategies_list=[]
    user_lines=[]
    user_utterances=[]
    for lines in file:
        
        if not lines.startswith(("overall","engaing","return","interupt","start")):
            if lines.startswith('Bot'):
                strategies_list.insert(len(strategies_list),check_strategies(lines))
            else:
                user_lines.append(get_utterance(lines))
                
        else:
            strategies_list=strategies_list[1:]
            if lines.endswith(','):
                populate_evaluation(lines)
                user_utterances=rows_utterance(user_lines)
                user_utterances=user_utterances[:len(user_utterances)-1]
                
                temp_dict[user_utterances[0]]=temp_dict[user_utterances[0]]+1
                R_table.at[user_utterances[0],strategies_list[0]]=((temp_dict[user_utterances[0]]-1)*R_table.at[user_utterances[0],strategies_list[0]]+0.8*float(evaluation_dict['start'])+0.2*float(evaluation_dict['overall']))/temp_dict[user_utterances[0]]
                count=1
                for line in user_utterances[1:len(user_utterances)]:
                    
                    temp_dict[user_utterances[count]]=temp_dict[user_utterances[count]]+1
                    R_table.at[user_utterances[count],strategies_list[count]]=((temp_dict[user_utterances[count]]-1)*R_table.at[user_utterances[count],strategies_list[count]]+0.4*float(evaluation_dict['engaing'])+0.4*float(evaluation_dict['interupt'])+0.2*float(evaluation_dict['overall']))/temp_dict[user_utterances[count]]               
                    count=count+1
            else:
                pass
        
    file.close()

In [ ]:
#for training from the model
for filename in os.listdir(abs_data_path):
    
    file = open(abs_data_path+filename,'r')
    strategies_list=[]
    user_lines=[]
    user_utterances=[]
    for lines in file:
        
        if not lines.startswith(("overall","engaing","return","interupt","start")):
            if lines.startswith('Bot'):
                strategies_list.insert(len(strategies_list),check_strategies(lines))
            else:
                user_lines.append(get_utterance(lines))
    user_utterances=rows_utterance(user_lines)
    strategies_list=strategies_list[1:]
    
    user_utterances=user_utterances[:len(user_utterances)-1]
    
    for values in user_utterances:
        indexes=user_utterances.index(values)
        training(indexes)

In [ ]:
N_TURN = random.choice(range(5,11)) 
DATA_PATH = abs_data_path+'/new_convo/'
strats = pickle.load(open(abs_strategies_path, "rb"))
strategies_list=[]
utterances=[]

def main():
    record_convo()
    while input("Do you want to record another conversation? (Y/N)").lower() == "y":
        record_convo()
        

def record_convo():
    # load strategies from pickle
    
    list1=[]
    try:
        # create file
        name = input("Before we start: what's your name?\n")
        date = str(datetime.datetime.today().date())
        convo_num = get_convo_num(name)
        filename = abs_data_path + "{0}_{1}_{2}.txt".format(name.lower(), date, convo_num) 
        
        f = open(filename, "w")
        
        # record converstaion
        print("----------------------------Here it gones--------------------------------")
        #strategy=Q_table.iloc[0].argmax()
        strategy = max_beginning()
        utterance = random.choice(strats[strategy])
        log("Bot", utterance, strategy, f)
        strats[strategy].remove(utterance)
        strategies_list.append(strategy)
        for turn in range(N_TURN-1):
            answer=str(input(name + ": "))
            log(name, answer, "None",f)
            
            utter=single_utterance(answer)
            strategy=''
            #for exploration
            if random.random()<epsilon:
                strategy = random.choice(list(strats.keys()))
            else:
                strategy=max_strategy(utter)
            utterances.append(utter)
            strategies_list.append(strategy)
            utterance=random.choice(list(strats[strategy]))
            log("Bot", utterance, strategy, f)
            strats[strategy].remove(utterance)
            
            utter=''
        answer=str(input(name + ": "))
        log(name, answer, "None",f) 
        #print(strategies_list)
        #print(utterances)
        for values in utterances:
            indexes=utterances.index(values)
            training(indexes)
        print("-----------------------------The end-------------------------------------")

        # evaluation and wrap up
        scores = get_evaluation()
        for score_type, score_value in scores.items():
            f.write("{0}={1},".format(score_type, score_value))
        f.close()
        print("Done!")
    
    except:
        os.remove(filename)
        raise    


def get_evaluation():
    scores = {}
    questions = {"overall": "How is the conveseration overall? (0 is aweful, 5 is amazing)\n",
                 "start": "How is the begining of the converstaion? (0: weird and out of context, 5: natural)\n",
                 "interupt": "How is the continuity of the conversation? (0: you get interupted too much, 5: very fluid and coherent)\n",
                 "engaing": "How engaging or interesting is the conversation? (0 is boring, 5 is very interesting)\n",
                 "return": "Would you like to talk to this bot again? (0 is not at all, 5 is definitely)\n"}
    for score_type in questions.keys():
        score = input(questions[score_type])
        while len(score) > 1 or ord(score) < ord("0") or ord(score) > ord("5"):
            score = input("Type a integer between [0,5], otherwise I will keep asking")
        scores[score_type] = score
    return scores


def get_convo_num(name):
    try:
        files = glob.glob(DATA_PATH + name.lower() + "_*")
        existing_nums = list(map(int, [i[:-4].split("_")[-1] for i in files]))
        return max(existing_nums) + 1
    except ValueError:
        return 1
    

def log(role, utter, strategy, file):
    if role == "Bot":
        print("Bot says: ", utter)
    time = datetime.datetime.now().strftime("%H:%M:%S")
    file.write("{0}_{1}_{2}:{3}\n".format(role, strategy, time, utter))

if __name__ == "__main__":
    
    main()